In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [2]:
client = bigquery.Client(project='dataproject-2-413010')

## QUERY

query_df = """
SELECT p.*, 
v.nombre_conductor, v.marca_coche, v.color_coche, v.anio_fabricacion, 
v.tipo_combustible, v.tiene_calefaccion, v.puntos_carnet, v.cilindraje_motor, v.kilometraje, v.anio_registro_app,
c.nombre_cliente, c.apellido_cliente, c.edad_cliente, c.genero_cliente,
c.direccion_cliente, c.trabajo_cliente, c.fecha_registro,
FROM dataproject-2-413010.BBDD.CochePole_BD p
left join dataproject-2-413010.BBDD.tabla_vehiculos v on p.vehiculo_id = v.vehiculo_id
left join dataproject-2-413010.BBDD.tabla_clientes c on p.cliente_id = c.cliente_id
"""

df = client.query(query_df).to_dataframe()

df.dtypes

cliente_id                         Int64
rating                           float64
metodo_pago                       object
pago_viaje                       float64
viaje_id                           Int64
vehiculo_id                        Int64
latitud                          float64
longitud                         float64
latitud_final                    float64
longitud_final                   float64
timestamp            datetime64[ns, UTC]
nombre_conductor                  object
marca_coche                       object
color_coche                       object
anio_fabricacion                   Int64
tipo_combustible                  object
tiene_calefaccion                boolean
puntos_carnet                      Int64
cilindraje_motor                   Int64
kilometraje                        Int64
anio_registro_app                  Int64
nombre_cliente                    object
apellido_cliente                  object
edad_cliente                       Int64
genero_cliente  

In [3]:
pd.set_option('display.max_columns', None)

df.head()

,cliente_id,rating,metodo_pago,pago_viaje,viaje_id,vehiculo_id,latitud,longitud,latitud_final,longitud_final,timestamp,nombre_conductor,marca_coche,color_coche,anio_fabricacion,tipo_combustible,tiene_calefaccion,puntos_carnet,cilindraje_motor,kilometraje,anio_registro_app,nombre_cliente,apellido_cliente,edad_cliente,genero_cliente,direccion_cliente,trabajo_cliente,fecha_registro
0,78947,8.0,Tarjeta,5.640651,18,108,39.50802,-0.32170,39.47947,-0.37594,2024-02-17 11:10:33.369962+00:00,Nicolás Ribera-Riba,BMW,Marrón siena,2020,Híbrido,True,11,2500,42353,2022,Evangelina,Valverde,48,mujer,"Vial de Apolonia Lpez 34 Piso 8 \nCeuta, 10701",Parquetero y colocador de suelos,2023-09-24
1,52772,7.0,Tarjeta,5.617617,18,848,39.50797,-0.32198,39.47947,-0.37594,2024-02-17 11:46:19.121628+00:00,Nazario Cantero Iniesta,BMW,Marrón cuero,2014,Eléctrico,True,13,0,119686,2023,Ligia,Rey,34,mujer,"Paseo Lorenza Vives 89 Puerta 5 \nHuelva, 47703",Gorjadore,2023-01-12
2,92572,6.0,Efectivo,5.609091,18,848,39.50797,-0.32210,39.47947,-0.37594,2024-02-17 11:47:49.767451+00:00,Nazario Cantero Iniesta,BMW,Marrón cuero,2014,Eléctrico,True,13,0,119686,2023,Pancho,Garzon,43,hombre,Urbanizacin de Manuel Lluch 932 Puerta 6 \nLle...,Sericultor,2023-08-01
3,47745,5.0,Tarjeta,5.619211,18,178,39.50795,-0.32194,39.47947,-0.37594,2024-02-17 11:56:18.100593+00:00,Valentín Quintana Clavero,Kia,Orquídea,2022,Híbrido,True,13,1600,14801,2023,Mariana,Mercader,40,mujer,"Calle de Vicenta Uriarte 61\nTarragona, 09066",Asesor financiero y en inversiones,2024-01-26
4,41370,9.0,Efectivo,1.797886,21,503,39.47026,-0.36855,39.46824,-0.38928,2024-02-17 10:54:05.960389+00:00,Guiomar Vaquero Sola,Subaru,Marrón rojizo,2020,Gasolina,False,10,1500,38659,2022,Borja,Miralles,33,hombre,"Acceso de Rolando Antn 86 Piso 6 \nLleida, 05124",Cartógrafo,2023-11-05


In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [5]:
columns_to_drop = ['pago_viaje', 'cliente_id', 'viaje_id', 'vehiculo_id', 'nombre_cliente', 'apellido_cliente', 'nombre_conductor', 'direccion_cliente']
X = df.drop(columns=columns_to_drop, axis=1)
y = df['pago_viaje']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Seleccionar las columnas numéricas, categóricas y booleanas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
boolean_features = X.select_dtypes(include=['boolean']).columns

# Crear transformadores para variables numéricas, categóricas y booleanas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

boolean_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Combinar transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ])

# Crear el modelo XGBoost
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Crear el pipeline con preprocesador y modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'model__n_estimators': [200, 300, 400],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [4, 5, 6],
    'model__subsample': [0.8, 0.9, 1.0],
    'model__colsample_bytree': [0.8, 0.9, 1.0],
}

# Configurar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Entrenar el modelo con búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo del grid search
best_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.3833648644280403


In [7]:
numeric_columns_after_transform = best_model.named_steps['preprocessor'].transformers_[0][1]['scaler'].get_feature_names_out(numeric_features)

categorical_columns_after_transform = best_model.named_steps['preprocessor'].transformers_[1][1]['onehot'].get_feature_names_out(categorical_features)

all_columns_after_transform = list(numeric_columns_after_transform) + list(categorical_columns_after_transform) + list(boolean_features)

X_train_transformed = best_model.named_steps['preprocessor'].transform(X_train)
X_train_transformed_df = pd.DataFrame(X_train_transformed.toarray(), columns=all_columns_after_transform)

variance_of_used_columns = X_train_transformed_df.var()
print('Varianza de las variables utilizadas:\n', variance_of_used_columns)


Varianza de las variables utilizadas:
 rating                                                         1.007042
latitud                                                        1.007042
longitud                                                       1.007042
latitud_final                                                  1.007042
longitud_final                                                 1.007042
                                                                 ...   
trabajo_cliente_Técnico en ingeniería de minas y metalurgia    0.006993
trabajo_cliente_Técnico en seguridad aeronáutica               0.006993
trabajo_cliente_Vendedor de comidas al mostrador               0.013888
trabajo_cliente_Vendedor por teléfono                          0.020684
tiene_calefaccion                                              0.250271
Length: 245, dtype: float64


In [8]:
variance_of_used_columns = X_train_transformed_df.var()

variance_of_used_columns_sorted = variance_of_used_columns.sort_values(ascending=True)

fig = px.bar(variance_of_used_columns_sorted, orientation='h', labels={'value': 'Varianza', 'index': 'Variables'},
             title='Varianza de las Variables Utilizadas (Ordenadas de Mayor a Menor)')

fig.show()

In [9]:
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rating', 'latitud', 'longitud', 'latitud_final', 'longitud_final',
       'anio_fabricacion', 'puntos_carnet', 'cilindraje_motor', 'kilometraje',
       'anio_registro_app', 'edad_cliente'],
      dtype='object')),
                                                 ('cat',
                                                  Pipelin...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [10]:
import joblib

joblib.dump(best_model, 'modelo_1.joblib')

['modelo_1.joblib']

In [11]:
cliente_data = {
    'rating': [8.0],
    'metodo_pago': ['Tarjeta'],
    'latitud': [39.50802],
    'longitud': [-0.32170],
    'latitud_final': [39.47947],
    'longitud_final': [-0.37594],
    'timestamp': ['2024-02-17 11:10:33.369962+00:00'],
    'marca_coche': ['BMW'],
    'color_coche': ['Marrón siena'],
    'anio_fabricacion': [2020],
    'tipo_combustible': ['Híbrido'],
    'tiene_calefaccion': [1],
    'puntos_carnet': [11],
    'cilindraje_motor': [2500],
    'kilometraje': [42353],
    'anio_registro_app': [2022],
    'edad_cliente': [48],
    'genero_cliente': ['mujer'],
    'trabajo_cliente': ['Parquetero y colocador de suelos'],
    'fecha_registro': ['2023-09-24']
}

cliente_df = pd.DataFrame(cliente_data)

prediction = best_model.predict(cliente_df)

print(f'Predicción de pago de viaje para el cliente: {prediction[0]}')


Predicción de pago de viaje para el cliente: 5.642274856567383
